<a href="https://colab.research.google.com/github/mukul27wno/Android-Studio/blob/main/recommendation_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Setup

In [14]:
#link with google drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
cd '/content/drive/MyDrive/Machine Learning/Mosahaya'

/content/drive/My Drive/Mukul/Machine Learning/Mosahaya


In [4]:
ls

 enrolled.csv                              'recommendation 1.ipynb'
'paraphrase-distilroberta-base-v1 cache'/


In [16]:
import pandas as pd #working with csv or excel files
import numpy as np #working with numbers/ arrays/ tensors
import tensorflow as tf #framework
from tensorflow import keras #API / library 
import os #using os commands between the python language

In [80]:
df=pd.read_csv("enrolled.csv") #load csv file as dataframe

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (6) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [33]:
df.tail(10)

,Unnamed: 0,Unnamed: 0.1,enrollmentId,generalData.name,generalData.age,generalData.mobileNumber,generalData.pinCode,skillData.education,skillData.preferredLocations,skillData.sectors,skillData.skills,first,second
176569,176569,176621,1.172120e+14,PRASANNA KUMAR SAHOO,30,9437807972,752120,secondary,"""anugul (odisha)"",""bhadrak (odisha)""","""others""","""others""","""anugul","(odisha)"",""bhadrak (odisha)"""
176570,176570,176639,7.039830e+15,K Bapuji,35,9178294775,760001,graduate,"""others""","""oil & gas""","""administration executive""","""others""",NaN
176571,176571,176645,5.126070e+15,Bulu Sha,24,9348971773,766011,iti & diploma,"""bhawanipatna (odisha)"",""bhubaneswar (odisha)""","""education & skill development""","""teacher (higher education)""","""bhawanipatna","(odisha)"",""bhubaneswar (odisha)"""
176572,176572,176649,6.563000e+15,prasanta Kumar behera,24,8018367433,753001,passed higher secondary (Class 12th),"""cuttack (odisha)""","""healthcare & pharmaceuticals"",""home services""...","""doctor (allopathic)"",""front line health worke...","""cuttack","(odisha)"""
176573,176573,176650,5.370140e+15,Siba Prasad Sahoo,25,9668814670,751016,graduate,"""bhubaneswar (odisha)""","""construction & real estate"",""professional & o...","""digital marketer"",""marketing executive"",""sale...","""bhubaneswar","(odisha)"""
176574,176574,176660,5.217870e+15,abhisek mohanty,27,7008723233,764020,graduate,"""bhubaneswar (odisha)""","""retail"",""tourism, hospitality & travel""","""retail cashier"",""retail operations assistant""...","""bhubaneswar","(odisha)"""
176575,176575,176678,5.761350e+15,Satyajeet Narayan,14,9337157292,769015,graduate,"""bangalore (karnataka)""","""professional & other services""","""astrologer""","""bangalore","(karnataka)"""
176576,176576,176686,7.475890e+15,Om Behera,21,8917681494,759020,iti & diploma,"""dhenkanal (odisha)""","""automobile"",""tourism, hospitality & travel""","""restaurant manager""","""dhenkanal","(odisha)"""
176577,176577,176687,2.485130e+15,Amarendra Kumar Das,52,9861255646,751002,graduate,"""bhubaneswar (odisha)""","""logistics & warehousing""","""cargo handler"",""warehouse associate"",""supply ...","""bhubaneswar","(odisha)"""
176578,176578,176701,1.364250e+15,Pratik Chandrakant Barge,21,8308720607,415004,graduate,"""pune (maharashtra)"",""greater mumbai (maharash...","""banking, financial services & insurance"",""edu...","""computer operator"",""data entry operator"",""dat...","""pune","(maharashtra)"",""greater mumbai (maharashtra)"",..."


In [34]:
df.head(10)

,Unnamed: 0,Unnamed: 0.1,enrollmentId,generalData.name,generalData.age,generalData.mobileNumber,generalData.pinCode,skillData.education,skillData.preferredLocations,skillData.sectors,skillData.skills,first,second
0,0,0,4.133420e+15,SOUMITRA BEHURA,29,9438266369,751003,graduate,"""bhubaneswar (odisha)""","""agriculture""","""poultry farmer""","""bhubaneswar","odisha"""
1,1,1,8.388130e+15,bijay sahoo,54,9867500133,752107,graduate,"""navi mumbai (maharashtra)""","""agriculture""","""dairy farmer""","""navi","mumbai maharashtra"""
2,2,2,2.134550e+15,Suvendu Nayak,29,7008803424,757041,graduate,"""jharsuguda (odisha)"",""talcher (odisha)""","""manufacturing & heavy engineering"",""others""","""others""","""jharsuguda","odisha"",""talcher odisha"""
3,3,3,2.167160e+15,Filan kumar patra,30,9556106091,752111,passed secondary (class 10th),"""navi mumbai (maharashtra)""","""construction & real estate""","""welder""","""navi","mumbai maharashtra"""
4,4,4,9.768460e+15,Santosh Kumar Patra,30,9040186174,751030,graduate,"""bhubaneswar (odisha)""","""banking, financial services & insurance""","""accountant""","""bhubaneswar","odisha"""
5,5,5,8.642260e+15,Kunal Kumar,32,7381596071,824122,passed higher secondary (Class 12th),"""jharsuguda (odisha)"",""talcher (odisha)""","""others""","""others""","""jharsuguda","odisha"",""talcher odisha"""
6,6,6,7.473310e+15,shubham yadav,27,8249467282,222142,graduate,"""talcher (odisha)"",""singrauli (madhya pradesh)""","""others""","""others""","""talcher","odisha"",""singrauli madhya pradesh"""
7,7,7,1.697560e+15,Jhulan Yadav,35,9661449162,841436,passed secondary (class 10th),"""jharsuguda (odisha)"",""talcher (odisha)"",""sing...","""mining""","""others""","""jharsuguda","odisha"",""talcher odisha"",""singrauli madhya pra..."
8,8,8,9.083130e+15,Manas Kumar Senapati,29,8895226314,757082,graduate,"""jharsuguda (odisha)""","""mining""","""others""","""jharsuguda","odisha"""
9,9,9,6.899290e+15,surendra munda,44,9939133981,829109,passed secondary (class 10th),"""jharsuguda (odisha)""","""mining""","""others""","""jharsuguda","odisha"""


#Data Processing

In [71]:
print(f'age: {df["generalData.age"].sort_values().unique()} \n')
print(f'pincode: {df["generalData.pinCode"].unique()} \n')
print(f'education: {df["skillData.education"].unique()} \n')
print(f'sectors: {df["skillData.sectors"].unique()} \n')
print(f'skills: {df["skillData.skills"].unique()} \n')

age: [  3   5   6   9  12  14  15  16  17  18  19  20  21  22  23  24  25  26
  27  28  29  30  31  32  33  34  35  36  37  38  39  40  41  42  43  44
  45  46  47  48  49  50  51  52  53  54  55  56  57  58  59  60  61  62
  64  65  66  69  70  72  73  77  79  80 100] 

pincode: [751003 752107 757041 ... '600113' '766011' '759020'] 

education: ['graduate' 'passed secondary (class 10th)'
 'passed higher secondary (Class 12th)' ... '!0th' 'BA Pass' '12th  Pass'] 

sectors: ['"agriculture"' '"manufacturing & heavy engineering","others"'
 '"construction & real estate"' ...
 '"healthcare & pharmaceuticals","home services","private security"'
 '"construction & real estate","professional & other services","others"'
 '"banking, financial services & insurance","education & skill development","it & ites","media & entertainment","telecommunication"'] 

skills: ['"poultry farmer"' '"dairy farmer"' '"others"' ... '"astrologer"'
 '"cargo handler","warehouse associate","supply chain manager"'
 '"co

In [83]:
print(f'There are {len(df.columns)} columnd in the dataframe df name')
print(df.columns)
print(df.shape)

There are 13 columnd in the dataframe df name
Index(['Unnamed: 0', 'Unnamed: 0.1', 'enrollmentId', 'generalData.name',
       'generalData.age', 'generalData.mobileNumber', 'generalData.pinCode',
       'skillData.education', 'skillData.preferredLocations',
       'skillData.sectors', 'skillData.skills', 'first', 'second'],
      dtype='object')
(176579, 13)


In [87]:
df=pd.read_csv("enrolled.csv")


def cleaner(df=df):
  states=[]
  locations=df["skillData.preferredLocations"].values
  import re #Regular expression operations

  for i in locations:
      res = re.findall(r'\(.*?\)',i)
      res=list(set(res))
      states.append(res)
      
  for i in states:
      for j in range((len(i))):
          i[j]=re.sub(r"[^a-zA-Z0-9]","",i[j])

  states_array=[]
  for i in states:
    str=""
    length=len(i)
    if length==1:
      str=str+i[0]
    if length>1:
      for j in i:
        str=str+", "+j
      str=str[2:]
    states_array.append(str)
  #----------------------------------------------------------------------------------------------------------------------------------------
  skills_array=[]
  skills= df["skillData.skills"].values
  for i in skills:
    i=i.replace('"', "")
    i=i.replace(",", ", ")
    skills_array.append(i)
  #----------------------------------------------------------------------------------------------------------------------------------------
  sectors_array=[]
  sectors= df["skillData.sectors"].values
  for i in sectors:
    i=i.replace('"', "")
    i=i.replace(",", ", ")
    sectors_array.append(i)

  df["skills"]=skills_array
  df["sectors"]=sectors_array
  df["states"]=states_array
  df=df.drop(["skillData.skills", "skillData.sectors"], axis=1)
  df=df.drop(["enrollmentId", "skillData.preferredLocations", "first", "second"], axis=1)
  return df

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (6) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [85]:
df=cleaner(df)

In [89]:
# Filling NaNs with empty string
df['skills'] = df['skills'].fillna('')
df['sectors'] = df['sectors'].fillna('')
df['states'] = df['states'].fillna('')
df.reset_index()
from sentence_transformers import SentenceTransformer

KeyError: ignored

In [90]:
model = SentenceTransformer('paraphrase-distilroberta-base-v1', cache_folder="E:\Mosahaya\paraphrase-distilroberta-base-v1 cache")

NameError: ignored

In [91]:
descriptions = df['sectors'].tolist()
# print(descriptions)
des_embeddings = []
c=0
for i,des in enumerate(descriptions):
    des_embeddings.append(model.encode(des))
    print(c)
    c+=1

KeyError: ignored

In [ ]:
import torch
from sentence_transformers import SentenceTransformer, util

def recommend(query):
    #Compute cosine-similarities with all embeddings 
    query_embedd = model.encode(query)
    cosine_scores = util.pytorch_cos_sim(query_embedd, des_embeddings)
    top5_matches = torch.argsort(cosine_scores, dim=1,descending=True).tolist()[0][1:20]
    return top5_matches

In [ ]:
sectors = 'agriculture'
#skills="farmer"
#states="odisha"
query_show_des = df.loc[(df['sectors'] == sectors)]['generalData.name'].to_list()[0]
recommendded_results = recommend(query_show_des)

indices = []
for index in recommendded_results:
    indices.append(index)

df = df.iloc[indices]
df